In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
driver = webdriver.Chrome(r"C:/chromedriver_win32/chromedriver.exe")

In [3]:
player_summary_df = pd.DataFrame(columns=["R", "player", "height", "weight", "apps",
                                          'mins','goals','Assists','Yel','Red',
                                          'SpG','PS%','AerialsWon','MotM','Rating'])

In [4]:
"""Need to write a dunction that mines all ids for all tournaments availible."""

#Get Basic Tournaments
def getPopularTournaments():
    comp_id = []
    comp_name =[]
    regions =[]
    #Create URL
    driver.get('https://www.whoscored.com')
    #Go to the wanted data table
    base = driver.find_element_by_id('popular')
    elements =base.find_elements_by_tag_name('li')
    #Get Ids
    comp_id.append([re.findall(r'\d+', elements[i].find_element_by_tag_name('a').get_attribute('href'))[1] for i in range(0,len(elements))])
    #Get Regions ID
    regions.append([re.findall(r'\d+', elements[i].find_element_by_tag_name('a').get_attribute('href'))[0] for i in range(0,len(elements))])
    #Get tournament name
    comp_name.append([elements[i].text for i in range(0,len(elements))])
    competitions = pd.DataFrame({'Name': comp_name[0],'ID': comp_id[0],'Region':regions[0]})
    return competitions

tournaments = getPopularTournaments()

In [11]:
tournaments

,Name,ID,Region
0,Premier League,2,252
1,Serie A,5,108
2,La Liga,4,206
3,Bundesliga,3,81
4,Ligue 1,22,74
5,Liga NOS,21,177
6,Eredivisie,13,155
7,Premier League,77,182
8,Brasileirão,95,31
9,Major League Soccer,85,233


In [9]:
#Get Teams Names and ids
def getTeams(tournaments):
    #I will include the europe tournaments here ( they have different page structure )
    tournaments = tournaments[0:15]
    ids = tournaments['ID'].values
    regions = tournaments['Region'].values
    teams = []
    links = []
    team_ids = []
    values=[]
    for i, r in zip(ids, regions):
        #Create URL
        driver.get('https://www.whoscored.com/Regions/'+str(r) + '/Tournaments/' + str(i))
        #Go to the wanted data table
        #base = driver.find_element_by_id('standings-16368-content')
        base = driver.find_element_by_class_name('standings')
        #Get table rows
        rows =base.find_elements_by_tag_name('tr')        
        for item in range(0,len(rows)):
            #Go to specific location to find the team name
            data = rows[item].find_elements_by_tag_name('td')
            teams.append(data[1].text)
            link = rows[item].find_elements_by_class_name('team-link')[0].get_attribute('href')
            team_id = re.search(r'\d+', link).group()
            values.append([i,data[1].text,team_id])
    teams = pd.DataFrame(values,columns=['Tournament_ID','Team_Name','Team_ID'])
    return teams
teams = getTeams(tournaments)

In [10]:
teams

,Tournament_ID,Team_Name,Team_ID
0,2,Liverpool,26
1,2,Manchester City,167
2,2,Tottenham,30
3,2,Arsenal,13
4,2,Manchester United,32
5,2,Chelsea,15
6,2,Wolverhampton Wanderers,161
7,2,Watford,27
8,2,West Ham,29
9,2,Leicester,14


In [10]:
"""Get Summary stats Stats Function"""
players_names = []
players_ids = []
def getPlayerSummary(teams):
    # Create empty list for future data
    values =[]
    for i in teams.values():
        #Get desired url
        driver.get('https://www.whoscored.com/Teams/'+str(i))
        #Wait for the url to load
        driver.implicitly_wait(40)
        base = driver.find_element_by_id('top-player-stats-summary-grid')
        table = base.find_element_by_id('player-table-statistics-body')
        tableRows = table.find_elements_by_tag_name('tr')
        for item in range(0,len(tableRows)):
                #Find the link for each player and extract the id from it
                link = tableRows[item].find_elements_by_class_name('player-link')[0].get_attribute('href')
                players_ids.append(re.search(r'\d+',link).group())
                row     = tableRows[item].find_elements_by_tag_name('td')
                players_names.append(row[2].text)
                values.append([row[i].text for i in range(1,len(row))])
    list_of_players = list(zip(players_names, players_ids))
    players = pd.DataFrame(list_of_players, columns = ['Name', 'ID'])
    dfOut = pd.DataFrame(values,columns=["R",'Name', "height", "weight", "apps",'mins','goals','Assists','Yel','Red',
                                         'SpG','PS%','AerialsWon','MotM','Rating'])
    dfOut = pd.merge(dfOut,players,on='Name')
    return dfOut
players_summary = getPlayerSummary(teams)

In [52]:
players_summary.head()

,R,Name,height,weight,apps,mins,goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating,ID
0,,"Mohamed Salah\n26, AM(CLR),FW",175,71,30(1),2649,17,7,-,-,3.4,76.8,0.3,6,7.52,108226
1,,"Virgil van Dijk\n27, D(C)",193,92,31,2755,3,1,1,-,0.8,89.6,4.7,4,7.44,95408
2,,"Sadio Mané\n26, AM(CLR),FW",175,69,28(1),2456,17,1,1,-,2.4,78,0.9,5,7.42,109915
3,,"Trent Alexander-Arnold\n20, D(R),M(R)",175,69,20(2),1874,1,6,3,-,1,77.2,0.3,-,7.22,318871
4,,"Dejan Lovren\n29, D(C)",188,84,9,810,1,-,1,-,0.6,85.8,4.6,1,7.21,29106


In [9]:
"""Get Defensive Stats Function"""

def getDefensiveStats(teams):
    #Create empty list for future data
    values=[]
    #set Url
    driver.get('https://www.whoscored.com/Teams/32/Show/England-Manchester-United')
    #Wait for the page to load.
    driver.implicitly_wait(40)
    #Get the right table by clicking the defensive tab on the data table.
    driver.find_elements_by_id("team-squad-stats-options")[0].find_elements_by_tag_name('li')[1].click()
    base = driver.find_elements_by_id('statistics-table-defensive')
    tableRows = base[0].find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row     = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values)
    #return the data table  
    return df
data = getDefensiveStats(teams)

In [18]:
"""Get Attacking Stats Function"""

def getAttackingStats(teams):
    #Create empty list for future data
    values=[]
    #set Url
    driver.get('https://www.whoscored.com/Teams/32/Show/England-Manchester-United')
    #Wait for the page to load.
    driver.implicitly_wait(40)
    #Get the right table by clicking the defensive tab on the data table.
    driver.find_elements_by_id("team-squad-stats-options")[0].find_elements_by_tag_name('li')[2].click()
    base = driver.find_elements_by_id('statistics-table-offensive')
    tableRows = base[0].find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row     = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values)
    #return the data table  
    return df
data = getAttackingStats(teams)